In [1]:
import numpy as np
import tensorflow as tf
import os
import tqdm
import glob
import sys
import matplotlib.pyplot as plt
import pickle
import multiprocessing
import itertools
import random
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
with open("train_files.pickle", "rb") as f:
    files_train = pickle.load(f)
with open("val_files.pickle", "rb") as f:
    files_val = pickle.load(f)

In [4]:
labels_train = []
phonemes_train = []
mfcc_train = []
video_train = []
for file in tqdm.tqdm(files_train):
    cur_labels = np.load(file)["labels"]
    cur_phonemes = np.load(file)["phonemes"]
    cur_mfcc = np.load(file)["mfcc"]
    cur_mfcc = (cur_mfcc - cur_mfcc.mean(axis=0))/cur_mfcc.std(axis=0)
    cur_video = np.load(file)["video"]
    labels_train.append(cur_labels)
    phonemes_train.append(cur_phonemes)
    mfcc_train.append(cur_mfcc)
    video_train.append(cur_video)

100%|██████████| 8433/8433 [01:10<00:00, 119.83it/s]


In [5]:
labels_val = []
phonemes_val = []
mfcc_val = []
video_val = []
for file in tqdm.tqdm(files_val):
    cur_labels = np.load(file)["labels"]
    cur_phonemes = np.load(file)["phonemes"]
    cur_mfcc = np.load(file)["mfcc"]
    cur_mfcc = (cur_mfcc - cur_mfcc.mean(axis=0))/cur_mfcc.std(axis=0)
    cur_video = np.load(file)["video"]
    labels_val.append(cur_labels)
    phonemes_val.append(cur_phonemes)
    mfcc_val.append(cur_mfcc)
    video_val.append(cur_video)

100%|██████████| 937/937 [00:08<00:00, 116.38it/s]


In [7]:
video_val[0].shape

(529, 30, 50)

In [6]:
def conv_layer(prev_layer, filter_width, num_filters, use_relu=True, use_batchnorn=True, is_training=None):
    convolution_out = tf.layers.conv1d(prev_layer, num_filters, filter_width, strides=1, padding="same", 
                                  activation=None)
    if use_batchnorn:
        if is_training is None:
            raise Exception("is_training placeholder required")
        convolution_out = tf.layers.batch_normalization(convolution_out, training=is_training)
    if use_relu:
        convolution_out = tf.nn.relu(convolution_out)
    return convolution_out

In [258]:
class PhonemeDNNModel:
    def __init__(self, num_features=(30, 50), num_symbols=23, optimizer="adam", use_batchnorm=True, folder="dnn_mfcc"):
        tf.reset_default_graph()
        self.num_features = num_features
        self.num_symbols = num_symbols
        self.checkpoints_folder = folder
        self.epoch = 0
        self.step = 0
        self.min_dev_loss = np.float('inf')
        self.global_step = tf.Variable(0, trainable=False, name="global_step")
        self.optimizer_type = optimizer
        self.use_batchnorm = use_batchnorm
        self._build_graph()

    def _create_placeholders(self):
        self.features_placeholder = tf.placeholder(tf.float32, [None, None, 30, 50, 1], name="features")
        self.features_len_placeholder = tf.placeholder(tf.int32, [None], name="features_len")
        self.target_placeholder = tf.placeholder(tf.int32, [None, None], name="targets")
        self.target_len_placeholder = tf.placeholder(tf.int32, [None], name="targets_len")

        self.learning_rate = tf.placeholder_with_default(1e-4, [], name="learning_rate")
        self.is_training = tf.placeholder_with_default(False, [], name="is_training")

        # transform target to sparse
#         target_ind = tf.where(tf.not_equal(tf.sequence_mask(self.target_len_placeholder), False))
#         target_val = tf.gather_nd(self.target_placeholder, target_ind)
#         self.target_sparse = tf.SparseTensor(target_ind, target_val,
#                                              tf.cast(tf.shape(self.target_placeholder), dtype=tf.int64))

    def _inference(self):
        # define architecture
        self.dropout = tf.cond(self.is_training, lambda: tf.constant(0.5), lambda: tf.constant(1.0))
        outputs = self.features_placeholder
        outputs = tf.layers.conv3d(outputs, filters=32, kernel_size=(3, 5, 5), activation=tf.nn.relu, padding="same")
        outputs = tf.layers.max_pooling3d(outputs, (1, 2, 2), (1, 2, 2), padding="same")
#         outputs = tf.nn.dropout(outputs, self.dropout)
        
        outputs = tf.layers.conv3d(outputs, filters=64, kernel_size=(3, 5, 5), activation=tf.nn.relu, padding="same")
        outputs = tf.layers.max_pooling3d(outputs, (1, 2, 2), (1, 2, 2), padding="same")
#         outputs = tf.nn.dropout(outputs, self.dropout)
        
        outputs = tf.layers.conv3d(outputs, filters=96, kernel_size=(3, 3, 3), activation=tf.nn.relu, padding="same")
        outputs = tf.layers.max_pooling3d(outputs, (1, 2, 2), (1, 2, 2), padding="same")
#         outputs = tf.nn.dropout(outputs, self.dropout)
        
        shape = tf.shape(outputs)
#         print(outputs)
        outputs = tf.reshape(outputs, (shape[0], shape[1], 4 * 7 * 96))
        
        with tf.variable_scope("rnn1"):
            cell1 = tf.nn.rnn_cell.GRUCell(256)
            outputs1, _ = tf.nn.bidirectional_dynamic_rnn(cell1, cell1, outputs, sequence_length=self.features_len_placeholder, dtype=tf.float32)
            outputs = tf.concat(outputs1, axis=-1)

        with tf.variable_scope("rnn2"):
            cell2 = tf.nn.rnn_cell.GRUCell(256)
            outputs2, _ = tf.nn.bidirectional_dynamic_rnn(cell2, 
                                            cell2, outputs, sequence_length=self.features_len_placeholder, dtype=tf.float32)
            outputs = tf.concat(outputs2, axis=-1)
        outputs = tf.layers.dense(outputs, self.num_symbols)
        logits_batch_major = outputs

        self.logits = logits_batch_major  # tf.transpose(logits_batch_major, [1, 0, 2])


    def _create_loss(self):
        self.predictions = tf.argmax(self.logits, axis=-1)
        self.predictions_prob = tf.nn.softmax(self.logits)
        mask = tf.sequence_mask(self.target_len_placeholder, dtype=tf.float32)
        self.accuracy = tf.reduce_sum(tf.cast(
            tf.equal(tf.cast(self.predictions, tf.int32), 
            self.target_placeholder), tf.float32) * mask) / tf.cast(tf.reduce_sum(self.target_len_placeholder), tf.float32)
        with tf.name_scope('training'):
            cost = tf.contrib.seq2seq.sequence_loss(self.logits, self.target_placeholder, mask)
            self.loss = tf.reduce_mean(cost, name='average_loss')


    def _create_optimizer(self):
        if self.optimizer_type == "adam":
            self.optimizer = tf.train.AdamOptimizer(self.learning_rate)
        elif self.optimizer_type == "yellowfin":
            raise ValueError("not now")
        else:
            raise ValueError("incorrect optimizer")

        gvs = self.optimizer.compute_gradients(self.loss)
        gradients, trainables = zip(*gvs)
        clipped_gradients, norm = tf.clip_by_global_norm(gradients, 5.0, name='clip_gradients')
        # batch normalization in tensorflow requires this extra dependency
        extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(extra_update_ops):
            self.train_step = self.optimizer.apply_gradients(zip(clipped_gradients, trainables),
                                                             global_step=self.global_step)


    def _build_graph(self):
        self._create_placeholders()
        self._inference()
        self._create_loss()
        self._network_vars = tf.global_variables()  # without optimizer
        self._create_optimizer()
        self.saver = tf.train.Saver()
        # self._create_summary()


    def save_weights(self, sess):
        self.saver.save(sess, 
            '/home/artbataev/Documents/checkpoints/{}/adam_checkpoints_best/ckpt'.format(self.checkpoints_folder), 
            self.step)


    def restore_weights(self, sess, only_network=False):
        ckpt = tf.train.get_checkpoint_state(
            '/home/artbataev/Documents/checkpoints/{}/adam_checkpoints_best/'.format(self.checkpoints_folder))
        if ckpt and ckpt.model_checkpoint_path:
            if only_network:
                saver = tf.train.Saver(self._network_vars)
                saver.restore(sess, ckpt.model_checkpoint_path)
            else:
                self.saver.restore(sess, ckpt.model_checkpoint_path)


    def init_op(self, sess, restore=True, restore_only_network=False):
        if restore:
            sess.run(tf.global_variables_initializer())
            self.restore_weights(sess, only_network=restore_only_network)
        else:
            sess.run(tf.global_variables_initializer())
    
    def get_predictions_p(self, features, features_len):
        feed_dict = {
                self.features_placeholder: features.reshape(1, features.shape[0], features.shape[1]),
                self.features_len_placeholder: features_len}
        pred_p = sess.run(self.predictions_prob, feed_dict=feed_dict)
        return pred_p[0]
    
    def test_net(self, batch_generator, sess, verbose=False):
        last = False
        full_len = 0
        loss = 0.0
        accuracy = 0.0
        while not last:
            texts, texts_len, features, features_len, last = next(batch_generator)
            feed_dict = {
                self.features_placeholder: features,
                self.features_len_placeholder: features_len,
                self.target_placeholder: texts,
                self.target_len_placeholder: texts_len
            }

            current_loss, current_acc = sess.run([self.loss, self.accuracy], feed_dict=feed_dict)

            loss += current_loss * len(texts)
            accuracy += current_acc * len(texts)
            full_len += len(texts)
        accuracy /= full_len
        loss /= full_len
        if verbose:
            print("Average test loss: {:.5f}".format(loss))
            print("Average test Accuracy: {:.2f}%".format(accuracy * 100))
        return loss, accuracy


    def run_step(self, batch_generator, sess, lr=1e-4):
        texts, texts_len, features, features_len, last = next(batch_generator)
#         print(features_len, texts_len)
        feed_dict = {
            self.features_placeholder: features,
            self.features_len_placeholder: features_len,
            self.target_placeholder: texts,
            self.target_len_placeholder: texts_len,
            self.is_training: True,
            self.learning_rate: lr,
        }

        current_loss, current_acc, _ = sess.run([self.loss, self.accuracy, self.train_step], feed_dict=feed_dict)
        self.step += 1
        if last:
            self.epoch += 1
            print("Epoch {}, end of dataset".format(self.epoch))
        return current_loss, current_acc


    def fit(self, batch_generator, dev_generator, sess, steps=1, lr=1e-4):
        try:
            train_loss = 0.0
            train_acc = 0.0
            for _ in range(steps):
                current_loss, current_acc = self.run_step(batch_generator, sess, lr)
                train_loss += current_loss
                train_acc += current_acc

                if self.step % 10 == 0:
                    train_loss /= 10
                    train_acc /= 10
                    print("Epoch {} step {} average_loss: {:.5f} Acc: {:.2f}%".format(
                        self.epoch, self.step, train_loss, train_acc * 100))
                else:
                    print(".", end="")

                if self.step % 50 == 0:
                    epoch_test_loss, epoch_test_acc = self.test_net(dev_generator, sess)
                    improved_text = "improved" if epoch_test_loss < self.min_dev_loss else "not improved"
                    print("Step {} test_loss: {:.5f} Acc: {:.2f}%".format(self.step, epoch_test_loss,
                                                                          np.mean(epoch_test_acc) * 100), improved_text)
                    if epoch_test_loss < self.min_dev_loss:
                        self.min_dev_loss = epoch_test_loss
                        self.save_weights(sess)

                if self.step % 10 == 0:
                    train_loss = 0.0
                    train_acc = 0.0


        except KeyboardInterrupt:
            print("Training Interrupted")

In [259]:
batch_size = 4

In [260]:
def train_generator():
    global batch_size
    while True:
        all_indices = np.random.choice(len(labels_train), len(labels_train), replace=False)
        for j in range(0, len(labels_train), batch_size):
            indices = all_indices[j: j + batch_size]
            texts_len = np.array(list(map(lambda x: len(x), (phonemes_train[i] for i in indices))))
            features_len = np.array(list(map(lambda x: len(x), (video_train[i] for i in indices))))
            texts = np.zeros((texts_len.shape[0], np.max(texts_len)), dtype=np.int)
            features = np.zeros((texts_len.shape[0], np.max(features_len), 30, 50))
            #         texts, texts_len, mfcc, mfcc_len, last
            for i in range(texts_len.shape[0]):
                texts[i, :len(phonemes_train[indices[i]])] = phonemes_train[indices[i]]
                features[i, :len(video_train[indices[i]])] = video_train[indices[i]]
            features = features[..., np.newaxis]
            yield texts, texts_len, features, features_len, j + batch_size >= len(labels_train)

In [261]:
def val_generator():
    batch_size = 8
#     global batch_size
    while True:
        for j in range(0, len(phonemes_val), batch_size):
            indices = np.arange(j, min(j + batch_size, len(phonemes_val)))
            texts_len = np.array(list(map(lambda x: len(x), (phonemes_val[i] for i in indices))))
            features_len = np.array(list(map(lambda x: len(x), (video_val[i] for i in indices))))
            texts = np.zeros((texts_len.shape[0], np.max(texts_len)), dtype=np.int)
            features = np.zeros((texts_len.shape[0], np.max(features_len), 30, 50))
            #         texts, texts_len, mfcc, mfcc_len, last
            for i in range(texts_len.shape[0]):
                texts[i, :len(phonemes_val[indices[i]])] = phonemes_val[indices[i]]
                features[i, :len(video_val[indices[i]])] = video_val[indices[i]]
            features = features[..., np.newaxis]
            yield texts, texts_len, features, features_len, (j + batch_size >= len(phonemes_val))

In [262]:
tgen = train_generator()
vgen = val_generator()

In [263]:
dnn = PhonemeDNNModel(optimizer="adam", use_batchnorm=False, folder="dnn_video")
sess = tf.Session()
dnn.init_op(sess, restore=False)

In [264]:
# dnn.run_step(tgen, sess)

In [ ]:
dnn.fit(tgen, vgen, sess, steps=1000)

.........Epoch 0 step 10 average_loss: 2.86819 Acc: 27.15%
.........Epoch 0 step 20 average_loss: 2.81652 Acc: 26.67%
.........Epoch 0 step 30 average_loss: 2.69050 Acc: 31.55%
.........Epoch 0 step 40 average_loss: 2.74946 Acc: 29.05%
.........Epoch 0 step 50 average_loss: 2.74001 Acc: 29.15%
Step 50 test_loss: 2.71822 Acc: 29.49% improved
.........Epoch 0 step 60 average_loss: 2.70765 Acc: 30.43%
.........Epoch 0 step 70 average_loss: 2.62562 Acc: 31.66%
.........Epoch 0 step 80 average_loss: 2.63696 Acc: 31.30%
.........Epoch 0 step 90 average_loss: 2.53085 Acc: 32.66%
.........Epoch 0 step 100 average_loss: 2.67588 Acc: 27.13%
Step 100 test_loss: 2.54257 Acc: 30.31% improved
.........Epoch 0 step 110 average_loss: 2.41080 Acc: 35.45%
.........Epoch 0 step 120 average_loss: 2.63856 Acc: 25.90%
.........Epoch 0 step 130 average_loss: 2.54365 Acc: 31.23%
.........Epoch 0 step 140 average_loss: 2.51406 Acc: 30.85%
.........Epoch 0 step 150 average_loss: 2.32929 Acc: 35.42%
Step 150 tes

In [44]:
files_test = list(glob.glob("../data/lip_reading/synchronized/test/*.npz"))

In [42]:
dnn = PhonemeDNNModel(optimizer="adam", use_batchnorm=True, folder="dnn_mfcc_norm") # 48 + 7x7 etc
sess = tf.Session()
dnn.init_op(sess, restore=True)

INFO:tensorflow:Restoring parameters from /home/artbataev/Documents/checkpoints/dnn_mfcc_norm/adam_checkpoints_best/ckpt-400


In [45]:
labels_test = []
mfcc_test = []
video_test = []
for file in tqdm.tqdm(files_test):
    labels_test.append(np.load(file)["labels"])
    cur_mfcc = np.load(file)["mfcc"]
    mfcc_test.append((cur_mfcc - cur_mfcc.mean(axis=0)) / cur_mfcc.std(axis=0))
    video_test.append(np.load(file)["video"])

100%|██████████| 2909/2909 [00:24<00:00, 121.12it/s]


In [46]:
predictions_path = "../data/lip_reading/3_predictions/mfcc_convdnn_2/"

In [47]:
for i, file in enumerate(files_test):
    predictions = dnn.get_predictions_p(mfcc_test[i], np.array(len(mfcc_test[i])).reshape(1,))
    np.save(predictions_path + os.path.splitext(os.path.basename(file))[0], predictions.astype(np.float32))
    print("processed: " + file + "(frames num: {})".format(predictions.shape[0]))

processed: ../data/lip_reading/synchronized/test/M0083_03_48790_Android_htc.npz(frames num: 150)
processed: ../data/lip_reading/synchronized/test/F0042_02_17538_Android_htc.npz(frames num: 278)
processed: ../data/lip_reading/synchronized/test/M0053_03_97513_Android_SM.npz(frames num: 299)
processed: ../data/lip_reading/synchronized/test/M0081_02_10694_Android_SM_Gal_J3.npz(frames num: 318)
processed: ../data/lip_reading/synchronized/test/M0047_01_69175_Android_htc.npz(frames num: 310)
processed: ../data/lip_reading/synchronized/test/M0016_02_1763092458_iPhone_Iphone5.npz(frames num: 826)
processed: ../data/lip_reading/synchronized/test/F0022_03_73890_Android_SM.npz(frames num: 414)
processed: ../data/lip_reading/synchronized/test/F0022_01_2637015489_iPhone_6s.npz(frames num: 519)
processed: ../data/lip_reading/synchronized/test/F0130_01_1935482760_Android_SM.npz(frames num: 580)
processed: ../data/lip_reading/synchronized/test/F0052_03_02864_iPhone_6s.npz(frames num: 187)
processed: ..

processed: ../data/lip_reading/synchronized/test/M0057_03_9801763425_iPhone_6s.npz(frames num: 550)
processed: ../data/lip_reading/synchronized/test/M0030_03_2107835946_iPhone_6s.npz(frames num: 536)
processed: ../data/lip_reading/synchronized/test/M0055_03_89462_Android_nexus.npz(frames num: 440)
processed: ../data/lip_reading/synchronized/test/F0034_02_95230_Android_htc.npz(frames num: 314)
processed: ../data/lip_reading/synchronized/test/F0031_02_1253068479_Android_htc.npz(frames num: 678)
processed: ../data/lip_reading/synchronized/test/F0009_02_2691345807_Android_htc.npz(frames num: 454)
processed: ../data/lip_reading/synchronized/test/F0021_02_42365_iPhone_iphone6.npz(frames num: 310)
processed: ../data/lip_reading/synchronized/test/M0058_01_48150_iPhone_6s.npz(frames num: 450)
processed: ../data/lip_reading/synchronized/test/F0119_02_86913_Android_nexus.npz(frames num: 331)
processed: ../data/lip_reading/synchronized/test/F0119_03_48721_Android_SM.npz(frames num: 280)
processed:

processed: ../data/lip_reading/synchronized/test/F0034_01_9872615403_iPhone_Iphone5.npz(frames num: 602)
processed: ../data/lip_reading/synchronized/test/M0065_03_51983_Android_htc.npz(frames num: 370)
processed: ../data/lip_reading/synchronized/test/F0052_02_89071_iPhone_iphone6.npz(frames num: 299)
processed: ../data/lip_reading/synchronized/test/M0001_02_1306459728_Android_SM.npz(frames num: 1003)
processed: ../data/lip_reading/synchronized/test/M0037_03_9015826743_Android_SM_Gal_J3.npz(frames num: 574)
processed: ../data/lip_reading/synchronized/test/F0020_03_42561_Android_SM.npz(frames num: 292)
processed: ../data/lip_reading/synchronized/test/M0041_02_42163_Android_SM.npz(frames num: 286)
processed: ../data/lip_reading/synchronized/test/F0034_03_5602743918_Android_SM_Gal_J3.npz(frames num: 708)
processed: ../data/lip_reading/synchronized/test/M0028_02_46783_Android_htc.npz(frames num: 342)
processed: ../data/lip_reading/synchronized/test/F0009_03_29748_iPhone_6s.npz(frames num: 2

processed: ../data/lip_reading/synchronized/test/F0197_01_27413_Android_SM_Gal_J3.npz(frames num: 452)
processed: ../data/lip_reading/synchronized/test/M0057_01_8102574936_iPhone_6s.npz(frames num: 799)
processed: ../data/lip_reading/synchronized/test/F0005_02_4067593812_Android_htc.npz(frames num: 514)
processed: ../data/lip_reading/synchronized/test/F0011_03_05849_Android_SM_Gal_J3.npz(frames num: 376)
processed: ../data/lip_reading/synchronized/test/F0010_02_1392408567_iPhone_iphone6.npz(frames num: 500)
processed: ../data/lip_reading/synchronized/test/F0073_01_47016_iPhone_6s.npz(frames num: 260)
processed: ../data/lip_reading/synchronized/test/F0042_02_1392408567_iPhone_iphone6.npz(frames num: 487)
processed: ../data/lip_reading/synchronized/test/M0060_03_51983_Android_htc.npz(frames num: 346)
processed: ../data/lip_reading/synchronized/test/F0172_01_7902154638_Android_SM.npz(frames num: 516)
processed: ../data/lip_reading/synchronized/test/M0077_03_60475_iPhone_6s.npz(frames num:

processed: ../data/lip_reading/synchronized/test/F0090_02_8054723916_Android_nexus.npz(frames num: 753)
processed: ../data/lip_reading/synchronized/test/M0057_03_03984_Android_SM.npz(frames num: 318)
processed: ../data/lip_reading/synchronized/test/M0033_01_41853_iPhone_Iphone5.npz(frames num: 360)
processed: ../data/lip_reading/synchronized/test/M0072_03_14590_Android_htc.npz(frames num: 370)
processed: ../data/lip_reading/synchronized/test/F0073_02_59637_Android_nexus.npz(frames num: 273)
processed: ../data/lip_reading/synchronized/test/F0032_02_4029763518_iPhone_iphone6.npz(frames num: 667)
processed: ../data/lip_reading/synchronized/test/F0042_03_73068_Android_SM_Gal_J3.npz(frames num: 216)
processed: ../data/lip_reading/synchronized/test/F0130_02_0439126578_Android_nexus.npz(frames num: 420)
processed: ../data/lip_reading/synchronized/test/F0010_01_63470_iPhone_6s.npz(frames num: 413)
processed: ../data/lip_reading/synchronized/test/F0201_01_7096245183_iPhone_6s.npz(frames num: 50

...

processed: ../data/lip_reading/synchronized/test/F0152_03_58601_Android_SM.npz(frames num: 222)
processed: ../data/lip_reading/synchronized/test/F0035_03_9263507184_Android_nexus.npz(frames num: 542)
processed: ../data/lip_reading/synchronized/test/F0052_02_05269_Android_htc.npz(frames num: 330)
processed: ../data/lip_reading/synchronized/test/M0028_03_74126_Android_nexus.npz(frames num: 344)
processed: ../data/lip_reading/synchronized/test/M0070_03_26738_Android_htc.npz(frames num: 314)
processed: ../data/lip_reading/synchronized/test/M0082_03_2738015469_iPhone_iphone6.npz(frames num: 686)
processed: ../data/lip_reading/synchronized/test/M0070_02_15238_Android_SM_Gal_J3.npz(frames num: 376)
processed: ../data/lip_reading/synchronized/test/M0055_01_87901_iPhone_6s.npz(frames num: 423)
processed: ../data/lip_reading/synchronized/test/F0173_01_87046_iPhone_6s.npz(frames num: 226)
processed: ../data/lip_reading/synchronized/test/F0009_01_1324670895_iPhone_6s.npz(frames num: 553)
processed

processed: ../data/lip_reading/synchronized/test/M0001_03_8415260937_Android_SM_Gal_J3.npz(frames num: 1310)
processed: ../data/lip_reading/synchronized/test/F0199_01_2837051964_Android_SM_Gal_J3.npz(frames num: 414)
processed: ../data/lip_reading/synchronized/test/F0010_03_3592601487_Android_SM.npz(frames num: 606)
processed: ../data/lip_reading/synchronized/test/F0201_02_89071_iPhone_iphone6.npz(frames num: 313)
processed: ../data/lip_reading/synchronized/test/F0069_02_97063_Android_nexus.npz(frames num: 222)
processed: ../data/lip_reading/synchronized/test/M0039_02_90864_Android_htc.npz(frames num: 398)
processed: ../data/lip_reading/synchronized/test/F0035_01_41853_iPhone_Iphone5.npz(frames num: 359)
processed: ../data/lip_reading/synchronized/test/F0052_03_32198_iPhone_6s.npz(frames num: 210)
processed: ../data/lip_reading/synchronized/test/F0022_02_9367254810_Android_htc.npz(frames num: 486)
processed: ../data/lip_reading/synchronized/test/M0030_03_1429067358_iPhone_6s.npz(frames

processed: ../data/lip_reading/synchronized/test/F0061_03_6382074195_iPhone_iphone6.npz(frames num: 662)
processed: ../data/lip_reading/synchronized/test/F0021_03_91605_iPhone_6s.npz(frames num: 375)
processed: ../data/lip_reading/synchronized/test/F0035_02_95230_Android_htc.npz(frames num: 378)
processed: ../data/lip_reading/synchronized/test/M0049_01_5031862974_iPhone_Iphone5.npz(frames num: 554)
processed: ../data/lip_reading/synchronized/test/M0001_03_5897106423_Android_SM_Gal_J3.npz(frames num: 1188)
processed: ../data/lip_reading/synchronized/test/M0012_03_45921_Android_SM_Gal_J3.npz(frames num: 248)
processed: ../data/lip_reading/synchronized/test/M0026_03_7130859426_Android_SM.npz(frames num: 420)
processed: ../data/lip_reading/synchronized/test/M0064_03_53061_iPhone_iphone6.npz(frames num: 314)
processed: ../data/lip_reading/synchronized/test/F0009_02_0582397146_Android_htc.npz(frames num: 474)
processed: ../data/lip_reading/synchronized/test/F0011_02_89071_iPhone_iphone6.npz(

processed: ../data/lip_reading/synchronized/test/M0002_03_9385741620_Android_SM_Gal_J3.npz(frames num: 580)
processed: ../data/lip_reading/synchronized/test/M0044_02_46783_Android_htc.npz(frames num: 362)
processed: ../data/lip_reading/synchronized/test/M0002_03_42950_Android_nexus.npz(frames num: 401)
processed: ../data/lip_reading/synchronized/test/F0032_02_1392408567_iPhone_iphone6.npz(frames num: 622)
processed: ../data/lip_reading/synchronized/test/F0095_01_65718_iPhone_6s.npz(frames num: 471)
processed: ../data/lip_reading/synchronized/test/F0020_02_95230_Android_htc.npz(frames num: 386)
processed: ../data/lip_reading/synchronized/test/M0041_01_75963_Android_htc.npz(frames num: 366)
processed: ../data/lip_reading/synchronized/test/M0079_01_89524_iPhone_6s.npz(frames num: 310)
processed: ../data/lip_reading/synchronized/test/F0090_03_35891_iPhone_6s.npz(frames num: 490)
processed: ../data/lip_reading/synchronized/test/F0175_02_68143_Android_nexus.npz(frames num: 228)
processed: ..

processed: ../data/lip_reading/synchronized/test/F0090_01_6271853904_iPhone_6s.npz(frames num: 846)
processed: ../data/lip_reading/synchronized/test/F0169_01_4923510687_Android_SM.npz(frames num: 881)
processed: ../data/lip_reading/synchronized/test/F0034_03_03582_Android_SM_Gal_J3.npz(frames num: 350)
processed: ../data/lip_reading/synchronized/test/F0021_02_2174905683_Android_htc.npz(frames num: 522)
processed: ../data/lip_reading/synchronized/test/F0172_03_18572_Android_nexus.npz(frames num: 216)
processed: ../data/lip_reading/synchronized/test/M0036_03_14358_Android_SM.npz(frames num: 356)
processed: ../data/lip_reading/synchronized/test/M0065_03_53061_iPhone_iphone6.npz(frames num: 313)
processed: ../data/lip_reading/synchronized/test/M0002_03_5873419602_Android_nexus.npz(frames num: 638)
processed: ../data/lip_reading/synchronized/test/M0089_02_8295164703_Android_nexus.npz(frames num: 779)
processed: ../data/lip_reading/synchronized/test/F0182_01_85213_iPhone_6s.npz(frames num: 3